# insightFace_Pytorch

In [58]:
from easydict import EasyDict as edict
from pathlib import Path, PureWindowsPath,PurePath
import torch
from torch.nn import CrossEntropyLoss
from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# from data.data_pipe import de_preprocess, get_train_loader, get_val_data
# from model import Backbone, Arcface, MobileFaceNet, Am_softmax, l2_norm
# from verifacation import evaluate
# from utils import get_time, gen_plot, hflip_batch, separate_bn_paras

import torch
from torch import optim
import numpy as np
from tqdm import tqdm
from tensorboardX import SummaryWriter
from matplotlib import pyplot as plt
plt.switch_backend('agg')
from PIL import Image
from torchvision import transforms as trans
import math
import bcolz

from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter
import torch.nn.functional as F
import torch
from collections import namedtuple
import math
import pdb

import numpy as np
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
import sklearn
from scipy import interpolate
import mxnet as mx
import io
import numpy as np
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
import sklearn
from scipy import interpolate
from datetime import datetime
import mxnet as mx



### utils.py

In [16]:
def separate_bn_paras(modules):
    if not isinstance(modules, list):
        modules = [*modules.modules()]
    paras_only_bn = []
    paras_wo_bn = []
    for layer in modules:
        if 'model' in str(layer.__class__):
            continue
        if 'container' in str(layer.__class__):
            continue
        else:
            if 'batchnorm' in str(layer.__class__):
                paras_only_bn.extend([*layer.parameters()])
            else:
                paras_wo_bn.extend([*layer.parameters()])
    return paras_only_bn, paras_wo_bn

def hflip_batch(imgs_tensor):
    hfliped_imgs = torch.empty_like(imgs_tensor)
    for i, img_ten in enumerate(imgs_tensor):
        hfliped_imgs[i] = hflip(img_ten)
    return hfliped_imgs

def get_time():
    return (str(datetime.now())[:-10]).replace(' ','-').replace(':','-')

def gen_plot(fpr, tpr):
    """Create a pyplot plot and save to buffer."""
    plt.figure()
    plt.xlabel("FPR", fontsize=14)
    plt.ylabel("TPR", fontsize=14)
    plt.title("ROC Curve", fontsize=14)
    plot = plt.plot(fpr, tpr, linewidth=2)
    buf = io.BytesIO()
    plt.savefig(buf, format='jpeg')
    buf.seek(0)
    plt.close()
    return buf


### data_pipe.py

In [83]:
def de_preprocess(tensor):
    return tensor*0.5 + 0.5
    
def get_train_dataset(imgs_folder):
    train_transform = trans.Compose([
        trans.RandomHorizontalFlip(),
        trans.ToTensor(),
        trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    ds = ImageFolder(imgs_folder, train_transform)
    class_num = ds[-1][1] + 1
    print('class_num', class_num)
    return ds, class_num

def get_train_loader(conf):
    if conf.data_mode in ['ms1m', 'concat']:
        ms1m_ds, ms1m_class_num = get_train_dataset(conf.ms1m_folder/'imgs')
        print('ms1m loader generated')
    if conf.data_mode in ['vgg', 'concat']:
        vgg_ds, vgg_class_num = get_train_dataset(conf.vgg_folder/'imgs')
        print('vgg loader generated')        
    if conf.data_mode == 'vgg':
        ds = vgg_ds
        class_num = vgg_class_num
    elif conf.data_mode == 'ms1m':
        ds = ms1m_ds
        class_num = ms1m_class_num
    elif conf.data_mode == 'concat':
        for i,(url,label) in enumerate(vgg_ds.imgs):
            vgg_ds.imgs[i] = (url, label + ms1m_class_num)
        ds = ConcatDataset([ms1m_ds,vgg_ds])
        class_num = vgg_class_num + ms1m_class_num
    elif conf.data_mode == 'emore':
        ds, class_num = get_train_dataset(conf.emore_folder/'imgs')
    elif conf.data_mode == 'small_vgg':
        ds, class_num = get_train_dataset(os.path.join(conf.smallvgg_folder,'imgs'))
        
    loader = DataLoader(ds, batch_size=conf.batch_size, shuffle=True, pin_memory=conf.pin_memory, num_workers=conf.num_workers)
    return loader, class_num 

def get_val_data(data_path):
#     agedb_30, agedb_30_issame = get_val_pair(data_path, 'agedb_30')
#     cfp_fp, cfp_fp_issame = get_val_pair(data_path, 'cfp_fp')
#     lfw, lfw_issame = get_val_pair(data_path, 'lfw')
#     return agedb_30, cfp_fp, lfw, agedb_30_issame, cfp_fp_issame, lfw_issame
#     lfw, lfw_issame = get_val_pair(data_path, 'lfw')
    lfw, lfw_issame = get_val_pair(data_path, 'small_lfw')

    return  lfw, lfw_issame

def get_val_pair(path, name):
    carray = bcolz.carray(rootdir = os.path.join(path,name), mode='r')
    issame = np.load(os.path.join(path,'{}_list.npy'.format(name)))
    return carray, issame

### model.py

In [84]:

##################################  Original Arcface Model #############################################################

class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

class SEModule(Module):
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = AdaptiveAvgPool2d(1)
        self.fc1 = Conv2d(
            channels, channels // reduction, kernel_size=1, padding=0 ,bias=False)
        self.relu = ReLU(inplace=True)
        self.fc2 = Conv2d(
            channels // reduction, channels, kernel_size=1, padding=0 ,bias=False)
        self.sigmoid = Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x

class bottleneck_IR(Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride ,bias=False), BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1 ,bias=False), PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1 ,bias=False), BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)
        return res + shortcut

class bottleneck_IR_SE(Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR_SE, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride ,bias=False), 
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3,3), (1,1),1 ,bias=False),
            PReLU(depth),
            Conv2d(depth, depth, (3,3), stride, 1 ,bias=False),
            BatchNorm2d(depth),
            SEModule(depth,16)
            )
    def forward(self,x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)
        return res + shortcut

class Bottleneck(namedtuple('Block', ['in_channel', 'depth', 'stride'])):
    '''A named tuple describing a ResNet block.'''
    
def get_block(in_channel, depth, num_units, stride = 2):
  return [Bottleneck(in_channel, depth, stride)] + [Bottleneck(depth, depth, 1) for i in range(num_units-1)]

def get_blocks(num_layers):
    if num_layers == 50:
        blocks = [
            get_block(in_channel=64, depth=64, num_units = 3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=14),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 100:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=13),
            get_block(in_channel=128, depth=256, num_units=30),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 152:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=8),
            get_block(in_channel=128, depth=256, num_units=36),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    return blocks

class Backbone(Module):
    def __init__(self, num_layers, drop_ratio, mode='ir'):
        super(Backbone, self).__init__()
        assert num_layers in [50, 100, 152], 'num_layers should be 50,100, or 152'
        assert mode in ['ir', 'ir_se'], 'mode should be ir or ir_se'
        blocks = get_blocks(num_layers)
        if mode == 'ir':
            unit_module = bottleneck_IR
        elif mode == 'ir_se':
            unit_module = bottleneck_IR_SE
        self.input_layer = Sequential(Conv2d(3, 64, (3, 3), 1, 1 ,bias=False), 
                                      BatchNorm2d(64), 
                                      PReLU(64))
        self.output_layer = Sequential(BatchNorm2d(512), 
                                       Dropout(drop_ratio),
                                       Flatten(),
                                       Linear(512 * 7 * 7, 512),
                                       BatchNorm1d(512))
        modules = []
        for block in blocks:
            for bottleneck in block:
                modules.append(
                    unit_module(bottleneck.in_channel,
                                bottleneck.depth,
                                bottleneck.stride))
        self.body = Sequential(*modules)
    
    def forward(self,x):
        x = self.input_layer(x)
        x = self.body(x)
        x = self.output_layer(x)
        return l2_norm(x)

##################################  MobileFaceNet #############################################################
    
class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class Depth_Wise(Module):
     def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)

class MobileFaceNet(Module):
    def __init__(self, embedding_size):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, embedding_size, bias=False)
        self.bn = BatchNorm1d(embedding_size)
    
    def forward(self, x):
        out = self.conv1(x)

        out = self.conv2_dw(out)

        out = self.conv_23(out)

        out = self.conv_3(out)
        
        out = self.conv_34(out)

        out = self.conv_4(out)

        out = self.conv_45(out)

        out = self.conv_5(out)

        out = self.conv_6_sep(out)

        out = self.conv_6_dw(out)

        out = self.conv_6_flatten(out)

        out = self.linear(out)

        out = self.bn(out)
        return l2_norm(out)

##################################  Arcface head #############################################################

class Arcface(Module):
    # implementation of additive margin softmax loss in https://arxiv.org/abs/1801.05599    
    def __init__(self, embedding_size=512, classnum=51332,  s=64., m=0.5):
        super(Arcface, self).__init__()
        self.classnum = classnum
        self.kernel = Parameter(torch.Tensor(embedding_size,classnum))
        # initial kernel
        self.kernel.data.uniform_(-1, 1).renorm_(2,1,1e-5).mul_(1e5)
        self.m = m # the margin value, default is 0.5
        self.s = s # scalar value default is 64, see normface https://arxiv.org/abs/1704.06369
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.mm = self.sin_m * m  # issue 1
        self.threshold = math.cos(math.pi - m)
    def forward(self, embbedings, label):
        # weights norm
        nB = len(embbedings)
        kernel_norm = l2_norm(self.kernel,axis=0)
        # cos(theta+m)
        cos_theta = torch.mm(embbedings,kernel_norm)
#         output = torch.mm(embbedings,kernel_norm)
        cos_theta = cos_theta.clamp(-1,1) # for numerical stability
        cos_theta_2 = torch.pow(cos_theta, 2)
        sin_theta_2 = 1 - cos_theta_2
        sin_theta = torch.sqrt(sin_theta_2)
        cos_theta_m = (cos_theta * self.cos_m - sin_theta * self.sin_m)
        # this condition controls the theta+m should in range [0, pi]
        #      0<=theta+m<=pi
        #     -m<=theta<=pi-m
        cond_v = cos_theta - self.threshold
        cond_mask = cond_v <= 0
        keep_val = (cos_theta - self.mm) # when theta not in [0,pi], use cosface instead
        cos_theta_m[cond_mask] = keep_val[cond_mask]
        output = cos_theta * 1.0 # a little bit hacky way to prevent in_place operation on cos_theta
        idx_ = torch.arange(0, nB, dtype=torch.long)
        output[idx_, label] = cos_theta_m[idx_, label]
        output *= self.s # scale up in order to make softmax work, first introduced in normface
        return output

##################################  Cosface head #############################################################    
    
class Am_softmax(Module):
    # implementation of additive margin softmax loss in https://arxiv.org/abs/1801.05599    
    def __init__(self,embedding_size=512,classnum=51332):
        super(Am_softmax, self).__init__()
        self.classnum = classnum
        self.kernel = Parameter(torch.Tensor(embedding_size,classnum))
        # initial kernel
        self.kernel.data.uniform_(-1, 1).renorm_(2,1,1e-5).mul_(1e5)
        self.m = 0.35 # additive margin recommended by the paper
        self.s = 30. # see normface https://arxiv.org/abs/1704.06369
    def forward(self,embbedings,label):
        kernel_norm = l2_norm(self.kernel,axis=0)
        cos_theta = torch.mm(embbedings,kernel_norm)
        cos_theta = cos_theta.clamp(-1,1) # for numerical stability
        phi = cos_theta - self.m
        label = label.view(-1,1) #size=(B,1)
        index = cos_theta.data * 0.0 #size=(B,Classnum)
        index.scatter_(1,label.data.view(-1,1),1)
        index = index.byte()
        output = cos_theta * 1.0
        output[index] = phi[index] #only change the correct predicted output
        output *= self.s # scale up in order to make softmax work, first introduced in normface
        return output



### verifications.py

def evaluate(embeddings, actual_issame, nrof_folds=10, pca=0):
    # Calculate evaluation metrics
    print('embeddings.shape',embeddings.shape)
    thresholds = np.arange(0, 4, 0.01)
    embeddings1 = embeddings[0::2]
    embeddings2 = embeddings[1::2]
    tpr, fpr, accuracy, best_thresholds = calculate_roc(thresholds, embeddings1, embeddings2,
                                       np.asarray(actual_issame), nrof_folds=nrof_folds, pca=pca)
#     thresholds = np.arange(0, 4, 0.001)
#     val, val_std, far = calculate_val(thresholds, embeddings1, embeddings2,
#                                       np.asarray(actual_issame), 1e-3, nrof_folds=nrof_folds)
#     return tpr, fpr, accuracy, best_thresholds, val, val_std, far
    return tpr, fpr, accuracy, best_thresholds

def calculate_roc(thresholds, embeddings1, embeddings2, actual_issame, nrof_folds=10, pca=0):
    assert (embeddings1.shape[0] == embeddings2.shape[0])
    assert (embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    print('nrof_pairs',nrof_pairs)
    nrof_thresholds = len(thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=False)

    tprs = np.zeros((nrof_folds, nrof_thresholds))
    fprs = np.zeros((nrof_folds, nrof_thresholds))
    accuracy = np.zeros((nrof_folds))
    best_thresholds = np.zeros((nrof_folds))
    indices = np.arange(nrof_pairs)
    # print('pca', pca)

    if pca == 0:
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff), 1)

    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        # print('train_set', train_set)
        # print('test_set', test_set)
        if pca > 0:
            print('doing pca on', fold_idx)
            embed1_train = embeddings1[train_set]
            embed2_train = embeddings2[train_set]
            _embed_train = np.concatenate((embed1_train, embed2_train), axis=0)
            # print(_embed_train.shape)
            pca_model = PCA(n_components=pca)
            pca_model.fit(_embed_train)
            embed1 = pca_model.transform(embeddings1)
            embed2 = pca_model.transform(embeddings2)
            embed1 = sklearn.preprocessing.normalize(embed1)
            embed2 = sklearn.preprocessing.normalize(embed2)
            # print(embed1.shape, embed2.shape)
            diff = np.subtract(embed1, embed2)
            dist = np.sum(np.square(diff), 1)

        # Find the best threshold for the fold
        acc_train = np.zeros((nrof_thresholds))
        for threshold_idx, threshold in enumerate(thresholds):
            _, _, acc_train[threshold_idx] = calculate_accuracy(threshold, dist[train_set], actual_issame[train_set])
        best_threshold_index = np.argmax(acc_train)
#         print('best_threshold_index', best_threshold_index, acc_train[best_threshold_index])
        best_thresholds[fold_idx] = thresholds[best_threshold_index]
        for threshold_idx, threshold in enumerate(thresholds):
            tprs[fold_idx, threshold_idx], fprs[fold_idx, threshold_idx], _ = calculate_accuracy(threshold,
                                                                                                 dist[test_set],
                                                                                                 actual_issame[
                                                                                                     test_set])
        _, _, accuracy[fold_idx] = calculate_accuracy(thresholds[best_threshold_index], dist[test_set],
                                                      actual_issame[test_set])

    tpr = np.mean(tprs, 0)
    fpr = np.mean(fprs, 0)
    return tpr, fpr, accuracy, best_thresholds

def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))

    tpr = 0 if (tp + fn == 0) else float(tp) / float(tp + fn)
    fpr = 0 if (fp + tn == 0) else float(fp) / float(fp + tn)
    acc = float(tp + tn) / dist.size
    return tpr, fpr, acc


### Learner.py

In [140]:
class face_learner(object):
    def __init__(self, conf, inference=False):
        print(conf)
        if conf.use_mobilfacenet:
            self.model = MobileFaceNet(conf.embedding_size).to(conf.device)
            print('MobileFaceNet model generated')
        else:
            self.model = Backbone(conf.net_depth, conf.drop_ratio, conf.net_mode).to(conf.device)
            print('{}_{} model generated'.format(conf.net_mode, conf.net_depth))
        
        
        if not inference:
            self.milestones = conf.milestones
            self.loader, self.class_num = get_train_loader(conf)   
            print('self.loader',self.loader)

            self.writer = SummaryWriter(conf.log_path)
            self.step = 0
            self.head = Arcface(embedding_size=conf.embedding_size, classnum=self.class_num).to(conf.device)

            print('two model heads generated')

            paras_only_bn, paras_wo_bn = separate_bn_paras(self.model)
        
            
            if conf.use_mobilfacenet:
                self.optimizer = optim.SGD([
                                    {'params': paras_wo_bn[:-1], 'weight_decay': 4e-5},
                                    {'params': [paras_wo_bn[-1]] + [self.head.kernel], 'weight_decay': 4e-4},
                                    {'params': paras_only_bn}
                                ], lr = conf.lr, momentum = conf.momentum)
            else:
                self.optimizer = optim.SGD([
                                    {'params': paras_wo_bn + [self.head.kernel], 'weight_decay': 5e-4},
#                                     {'params': paras_only_bn}
                                ], lr = conf.lr, momentum = conf.momentum)
            print(self.optimizer)
#             self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=40, verbose=True)

            print('optimizers generated')    
#             self.board_loss_every = len(self.loader)//100
#             self.evaluate_every = len(self.loader)//10
#             self.save_every = len(self.loader)//5
           
            self.board_loss_every = len(self.loader)
            self.evaluate_every = len(self.loader)
            self.save_every = len(self.loader)
#             self.agedb_30, self.cfp_fp, self.lfw, self.agedb_30_issame, self.cfp_fp_issame, self.lfw_issame = get_val_data(self.loader.dataset.root.parent)
            self.lfw, self.lfw_issame = get_val_data(conf.smallvgg_folder)

#         else:
            self.threshold = conf.threshold
    
    def save_state(self, conf, accuracy, to_save_folder=False, extra=None, model_only=False):
        if to_save_folder:
            save_path = conf.save_path
        else:
            save_path = conf.model_path
            
        torch.save(
            model.state_dict(), 
            save_path('model_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))
        if not model_only:
            torch.save(
                head.state_dict(), save_path('head_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))
            torch.save(
                optimizer.state_dict(), save_path('optimizer_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))
        
    def load_state(self, conf, fixed_str, from_save_folder=False, model_only=False):
        if from_save_folder:
            save_path = conf.save_path
        else:
            save_path = conf.model_path            
        self.model.load_state_dict(torch.load(save_path/'model_{}'.format(fixed_str)))
        if not model_only:
            self.head.load_state_dict(torch.load(save_path/'head_{}'.format(fixed_str)))
            self.optimizer.load_state_dict(torch.load(save_path/'optimizer_{}'.format(fixed_str)))
        
    def board_val(self, db_name, accuracy, best_threshold, roc_curve_tensor):
        self.writer.add_scalar('{}_accuracy'.format(db_name), accuracy, self.step)
        self.writer.add_scalar('{}_best_threshold'.format(db_name), best_threshold, self.step)
        self.writer.add_image('{}_roc_curve'.format(db_name), roc_curve_tensor, self.step)
#         self.writer.add_scalar('{}_val:true accept ratio'.format(db_name), val, self.step)
#         self.writer.add_scalar('{}_val_std'.format(db_name), val_std, self.step)
#         self.writer.add_scalar('{}_far:False Acceptance Ratio'.format(db_name), far, self.step)
        
    def evaluate(self, conf, carray, issame, nrof_folds = 5, tta = False):
        self.model.eval()
        idx = 0
        embeddings = np.zeros([len(carray), conf.embedding_size])
        with torch.no_grad():
            while idx + conf.batch_size <= len(carray):
                batch = torch.tensor(carray[idx:idx + conf.batch_size])
                if tta:
                    fliped = hflip_batch(batch)
                    emb_batch = self.model(batch.to(conf.device)) + self.model(fliped.to(conf.device))
                    embeddings[idx:idx + conf.batch_size] = l2_norm(emb_batch)
                else:
                    embeddings[idx:idx + conf.batch_size] = self.model(batch.to(conf.device)).cpu()
                idx += conf.batch_size
            if idx < len(carray):
                batch = torch.tensor(carray[idx:])            
                if tta:
                    fliped = hflip_batch(batch)
                    emb_batch = self.model(batch.to(conf.device)) + self.model(fliped.to(conf.device))
                    embeddings[idx:] = l2_norm(emb_batch)
                else:
                    embeddings[idx:] = self.model(batch.to(conf.device)).cpu()
        tpr, fpr, accuracy, best_thresholds = evaluate(embeddings, issame, nrof_folds)
        buf = gen_plot(fpr, tpr)
        roc_curve = Image.open(buf)
        roc_curve_tensor = trans.ToTensor()(roc_curve)
        return accuracy.mean(), best_thresholds.mean(), roc_curve_tensor
    
    def find_lr(self,
                conf,
                init_value=1e-8,
                final_value=10.,
                beta=0.98,
                bloding_scale=3.,
                num=None):
        if not num:
            num = len(self.loader)
        mult = (final_value / init_value)**(1 / num)
        lr = init_value
        for params in self.optimizer.param_groups:
            params['lr'] = lr
        self.model.train()
        avg_loss = 0.
        best_loss = 0.
        batch_num = 0
        losses = []
        log_lrs = []
        for i, (imgs, labels) in tqdm(enumerate(self.loader), total=num):

            imgs = imgs.to(conf.device)
            labels = labels.to(conf.device)
            batch_num += 1          

            self.optimizer.zero_grad()

            embeddings = self.model(imgs)
            thetas = self.head(embeddings, labels)
            loss = conf.ce_loss(thetas, labels)          
          
            #Compute the smoothed loss
            avg_loss = beta * avg_loss + (1 - beta) * loss.item()
            self.writer.add_scalar('avg_loss', avg_loss, batch_num)
            smoothed_loss = avg_loss / (1 - beta**batch_num)
            self.writer.add_scalar('smoothed_loss', smoothed_loss,batch_num)
            #Stop if the loss is exploding
            if batch_num > 1 and smoothed_loss > bloding_scale * best_loss:
                print('exited with best_loss at {}'.format(best_loss))
                plt.plot(log_lrs[10:-5], losses[10:-5])
                return log_lrs, losses
            #Record the best loss
            if smoothed_loss < best_loss or batch_num == 1:
                best_loss = smoothed_loss
            #Store the values
            losses.append(smoothed_loss)
            log_lrs.append(math.log10(lr))
            self.writer.add_scalar('log_lr', math.log10(lr), batch_num)
            #Do the SGD step
            #Update the lr for the next step

            loss.backward()
            self.optimizer.step()

            lr *= mult
            for params in self.optimizer.param_groups:
                params['lr'] = lr
            if batch_num > num:
                plt.plot(log_lrs[10:-5], losses[10:-5])
                return log_lrs, losses    

In [109]:
class face_learner(object):
    def __init__(self, conf, inference=False):
        print(conf)
        if conf.use_mobilfacenet:
            self.model = MobileFaceNet(conf.embedding_size).to(conf.device)
            print('MobileFaceNet model generated')
        else:
            self.model = Backbone(conf.net_depth, conf.drop_ratio, conf.net_mode).to(conf.device)
            print('{}_{} model generated'.format(conf.net_mode, conf.net_depth))
        
        
        if not inference:
            self.milestones = conf.milestones
            self.loader, self.class_num = get_train_loader(conf)   
            print('self.loader',self.loader)

            self.writer = SummaryWriter(conf.log_path)
            self.step = 0
            self.head = Arcface(embedding_size=conf.embedding_size, classnum=self.class_num).to(conf.device)

            print('two model heads generated')

            paras_only_bn, paras_wo_bn = separate_bn_paras(self.model)
        
            
            if conf.use_mobilfacenet:
                self.optimizer = optim.SGD([
                                    {'params': paras_wo_bn[:-1], 'weight_decay': 4e-5},
                                    {'params': [paras_wo_bn[-1]] + [self.head.kernel], 'weight_decay': 4e-4},
                                    {'params': paras_only_bn}
                                ], lr = conf.lr, momentum = conf.momentum)
            else:
                self.optimizer = optim.SGD([
                                    {'params': paras_wo_bn + [self.head.kernel], 'weight_decay': 5e-4},
#                                     {'params': paras_only_bn}
                                ], lr = conf.lr, momentum = conf.momentum)
            print(self.optimizer)
#             self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=40, verbose=True)

            print('optimizers generated')    
#             self.board_loss_every = len(self.loader)//100
#             self.evaluate_every = len(self.loader)//10
#             self.save_every = len(self.loader)//5
           
            self.board_loss_every = len(self.loader)
            self.evaluate_every = len(self.loader)
            self.save_every = len(self.loader)
#             self.agedb_30, self.cfp_fp, self.lfw, self.agedb_30_issame, self.cfp_fp_issame, self.lfw_issame = get_val_data(self.loader.dataset.root.parent)
            self.lfw, self.lfw_issame = get_val_data(conf.smallvgg_folder)

#         else:
            self.threshold = conf.threshold
    
    def save_state(self, conf, accuracy, to_save_folder=False, extra=None, model_only=False):
        if to_save_folder:
            save_path = conf.save_path
        else:
            save_path = conf.model_path
        torch.save(
            self.model.state_dict(), save_path /
            ('model_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, self.step, extra)))
        if not model_only:
            torch.save(
                self.head.state_dict(), save_path /
                ('head_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, self.step, extra)))
            torch.save(
                self.optimizer.state_dict(), save_path /
                ('optimizer_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, self.step, extra)))
    
    def load_state(self, conf, fixed_str, from_save_folder=False, model_only=False):
        if from_save_folder:
            save_path = conf.save_path
        else:
            save_path = conf.model_path            
        self.model.load_state_dict(torch.load(save_path/'model_{}'.format(fixed_str)))
        if not model_only:
            self.head.load_state_dict(torch.load(save_path/'head_{}'.format(fixed_str)))
            self.optimizer.load_state_dict(torch.load(save_path/'optimizer_{}'.format(fixed_str)))
        
    def board_val(self, db_name, accuracy, best_threshold, roc_curve_tensor):
        self.writer.add_scalar('{}_accuracy'.format(db_name), accuracy, self.step)
        self.writer.add_scalar('{}_best_threshold'.format(db_name), best_threshold, self.step)
        self.writer.add_image('{}_roc_curve'.format(db_name), roc_curve_tensor, self.step)
#         self.writer.add_scalar('{}_val:true accept ratio'.format(db_name), val, self.step)
#         self.writer.add_scalar('{}_val_std'.format(db_name), val_std, self.step)
#         self.writer.add_scalar('{}_far:False Acceptance Ratio'.format(db_name), far, self.step)
        
    def evaluate(self, conf, carray, issame, nrof_folds = 5, tta = False):
        self.model.eval()
        idx = 0
        embeddings = np.zeros([len(carray), conf.embedding_size])
        with torch.no_grad():
            while idx + conf.batch_size <= len(carray):
                batch = torch.tensor(carray[idx:idx + conf.batch_size])
                if tta:
                    fliped = hflip_batch(batch)
                    emb_batch = self.model(batch.to(conf.device)) + self.model(fliped.to(conf.device))
                    embeddings[idx:idx + conf.batch_size] = l2_norm(emb_batch)
                else:
                    embeddings[idx:idx + conf.batch_size] = self.model(batch.to(conf.device)).cpu()
                idx += conf.batch_size
            if idx < len(carray):
                batch = torch.tensor(carray[idx:])            
                if tta:
                    fliped = hflip_batch(batch)
                    emb_batch = self.model(batch.to(conf.device)) + self.model(fliped.to(conf.device))
                    embeddings[idx:] = l2_norm(emb_batch)
                else:
                    embeddings[idx:] = self.model(batch.to(conf.device)).cpu()
        tpr, fpr, accuracy, best_thresholds = evaluate(embeddings, issame, nrof_folds)
        buf = gen_plot(fpr, tpr)
        roc_curve = Image.open(buf)
        roc_curve_tensor = trans.ToTensor()(roc_curve)
        return accuracy.mean(), best_thresholds.mean(), roc_curve_tensor
    
    def find_lr(self,
                conf,
                init_value=1e-8,
                final_value=10.,
                beta=0.98,
                bloding_scale=3.,
                num=None):
        if not num:
            num = len(self.loader)
        mult = (final_value / init_value)**(1 / num)
        lr = init_value
        for params in self.optimizer.param_groups:
            params['lr'] = lr
        self.model.train()
        avg_loss = 0.
        best_loss = 0.
        batch_num = 0
        losses = []
        log_lrs = []
        for i, (imgs, labels) in tqdm(enumerate(self.loader), total=num):

            imgs = imgs.to(conf.device)
            labels = labels.to(conf.device)
            batch_num += 1          

            self.optimizer.zero_grad()

            embeddings = self.model(imgs)
            thetas = self.head(embeddings, labels)
            loss = conf.ce_loss(thetas, labels)          
          
            #Compute the smoothed loss
            avg_loss = beta * avg_loss + (1 - beta) * loss.item()
            self.writer.add_scalar('avg_loss', avg_loss, batch_num)
            smoothed_loss = avg_loss / (1 - beta**batch_num)
            self.writer.add_scalar('smoothed_loss', smoothed_loss,batch_num)
            #Stop if the loss is exploding
            if batch_num > 1 and smoothed_loss > bloding_scale * best_loss:
                print('exited with best_loss at {}'.format(best_loss))
                plt.plot(log_lrs[10:-5], losses[10:-5])
                return log_lrs, losses
            #Record the best loss
            if smoothed_loss < best_loss or batch_num == 1:
                best_loss = smoothed_loss
            #Store the values
            losses.append(smoothed_loss)
            log_lrs.append(math.log10(lr))
            self.writer.add_scalar('log_lr', math.log10(lr), batch_num)
            #Do the SGD step
            #Update the lr for the next step

            loss.backward()
            self.optimizer.step()

            lr *= mult
            for params in self.optimizer.param_groups:
                params['lr'] = lr
            if batch_num > num:
                plt.plot(log_lrs[10:-5], losses[10:-5])
                return log_lrs, losses    

    def train(self, conf, epochs):
        self.model.train()
        
        running_loss = 0.            
        for e in range(epochs):
            print('epoch {} started'.format(e))
            
            # schdule_lr: lr을 1/10으로 나누기
            if e == self.milestones[0]:
                self.schedule_lr()
            if e == self.milestones[1]:
                self.schedule_lr()      
            if e == self.milestones[2]:
                self.schedule_lr()    
                
            for imgs, labels in tqdm(iter(self.loader)):
                imgs = imgs.to(conf.device)
                labels = labels.to(conf.device)
                self.optimizer.zero_grad()
                embeddings = self.model(imgs)
                thetas = self.head(embeddings, labels)
                loss = conf.ce_loss(thetas, labels)
                loss.backward()
                running_loss += loss.item()
                self.optimizer.step()
                
                print('self.board_loss_every',self.board_loss_every)
                if self.step % self.board_loss_every == 0 and self.step != 0:
                    loss_board = running_loss / self.board_loss_every
                    self.writer.add_scalar('train_loss', loss_board, self.step)
                    running_loss = 0.
                
                if self.step % self.evaluate_every == 0 and self.step != 0:
#                     accuracy, best_threshold, roc_curve_tensor = self.evaluate(conf, self.agedb_30, self.agedb_30_issame)
#                     self.board_val('agedb_30', accuracy, best_threshold, roc_curve_tensor)
                    accuracy, best_threshold, roc_curve_tensor = self.evaluate(conf, self.lfw, self.lfw_issame)
                    self.board_val('lfw', accuracy, best_threshold, roc_curve_tensor)
#                     accuracy, best_threshold, roc_curve_tensor = self.evaluate(conf, self.cfp_fp, self.cfp_fp_issame)
#                     self.board_val('cfp_fp', accuracy, best_threshold, roc_curve_tensor)
                    self.model.train()
                if self.step % self.save_every == 0 and self.step != 0:
                    self.save_state(conf, accuracy)
                    
                self.step += 1
                
        self.save_state(conf, accuracy, to_save_folder=True, extra='final')

    def schedule_lr(self):
        print('self.optimizer.param_groups',self.optimizer.param_groups)
        for params in self.optimizer.param_groups:                 
            params['lr'] /= 10
        print(self.optimizer)
    
    def infer(self, conf, faces, target_embs, tta=False):
        '''
        faces : list of PIL Image
        target_embs : [n, 512] computed embeddings of faces in facebank
        names : recorded names of faces in facebank
        tta : test time augmentation (hfilp, that's all)
        '''
        embs = []
        for img in faces:
            if tta:
                mirror = trans.functional.hflip(img)
                emb = self.model(conf.test_transform(img).to(conf.device).unsqueeze(0))
                emb_mirror = self.model(conf.test_transform(mirror).to(conf.device).unsqueeze(0))
                embs.append(l2_norm(emb + emb_mirror))
            else:                        
                embs.append(self.model(conf.test_transform(img).to(conf.device).unsqueeze(0)))
        source_embs = torch.cat(embs)
        
        diff = source_embs.unsqueeze(-1) - target_embs.transpose(1,0).unsqueeze(0)
        dist = torch.sum(torch.pow(diff, 2), dim=1)
        minimum, min_idx = torch.min(dist, dim=1)
        min_idx[minimum > self.threshold] = -1 # if no match, set idx to -1
        return min_idx, minimum               

### 기본 적인 변수들: config.py

In [222]:
def get_config(training = True):
    conf = edict()
#     conf.data_path = Path('../data')
#     conf.work_path = PureWindowsPath('./work_space/')
#     conf.model_path =conf.work_path/'models'
#     conf.log_path = conf.work_path/'log'
#     conf.save_path = conf.work_path/'save'
    conf.data_path = '../data/'
    conf.work_path = './work_space/'
    conf.model_path  = os.path.join(conf.work_path,'models/')
    conf.log_path = os.path.join(conf.work_path,'log/')
    conf.save_path = os.path.join(conf.work_path,'save/')
    conf.input_size = [112,112]
    conf.embedding_size = 512
    conf.use_mobilfacenet = False
    conf.net_depth = 50
    conf.drop_ratio = 0.6
    conf.net_mode = 'ir_se' # or 'ir'
    conf.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    conf.test_transform = trans.Compose([
                    trans.ToTensor(),
                    trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                ])
    conf.data_mode = 'emore'
    conf.smallvgg_folder = os.path.join(conf.data_path,'small_vgg')
    conf.vgg_folder = os.path.join(conf.data_path,'faces_vgg2_112x112')
#     conf.ms1m_folder = conf.data_path/'faces_ms1m_112x112'
#     conf.emore_folder = conf.data_path/'faces_emore'
    conf.batch_size = 100 # irse net depth 50 
#   conf.batch_size = 200 # mobilefacenet
    conf.batch_size = 32 # small_vgg

#--------------------Training Config ------------------------    
    if training:        
        conf.log_path = os.path.join(conf.work_path,'log')
        conf.save_path = os.path.join(conf.work_path,'save')
    #     conf.weight_decay = 5e-4
        conf.lr = 1e-3
        conf.milestones = [12,15,18]
        conf.momentum = 0.9
        conf.pin_memory = True
#         conf.num_workers = 4 # when batchsize is 200
        conf.num_workers = 3
        conf.ce_loss = CrossEntropyLoss()    
        conf.threshold = 1.5
#--------------------Inference Config ------------------------
    else:
        conf.facebank_path = conf.data_path/'facebank'
        conf.threshold = 1.5
        conf.face_limit = 10 
        #when inference, at maximum detect 10 faces in one image, my laptop is slow
        conf.min_face_size = 30 
        # the larger this value, the faster deduction, comes with tradeoff in small faces
    return conf

## main()

In [223]:
args = edict()

args.epochs =20
args.net_mode = 'ir_se'  #"which network, [ir, ir_se, mobilefacenet]"
args.net_depth = 50 # "how many layers [50,100,152]"
args.lr = 1e-3
args.batch_size = 32
args.num_workers = 3
args.data_mode = 'small_vgg' # use which database, [vgg, ms1m, emore, concat]"

conf = get_config()

if args.net_mode == 'mobilefacenet':
        conf.use_mobilfacenet = True
else:
    conf.net_mode = args.net_mode
    conf.net_depth = args.net_depth    

    
conf.lr = args.lr
conf.batch_size = args.batch_size
conf.num_workers = args.num_workers
conf.data_mode = args.data_mode

In [224]:
learner = face_learner(conf)

{'data_path': '../data/', 'work_path': './work_space/', 'model_path': './work_space/models/', 'log_path': './work_space/log', 'save_path': './work_space/save', 'input_size': [112, 112], 'embedding_size': 512, 'use_mobilfacenet': False, 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cpu'), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'small_vgg', 'smallvgg_folder': '../data/small_vgg', 'vgg_folder': '../data/faces_vgg2_112x112', 'batch_size': 32, 'lr': 0.001, 'milestones': [12, 15, 18], 'momentum': 0.9, 'pin_memory': True, 'num_workers': 3, 'ce_loss': CrossEntropyLoss(), 'threshold': 1.5}
ir_se_50 model generated
class_num 3
self.loader <torch.utils.data.dataloader.DataLoader object at 0x000001DCFEC532C8>
two model heads generated
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
optimizers generated


In [225]:
learner.model

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): bottleneck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): PReLU(num_parameters=64)
        (3)

In [226]:
#learner.train(conf, args.epochs)

# learner.train분석

In [227]:
inference=False
step =0
print(conf)

{'data_path': '../data/', 'work_path': './work_space/', 'model_path': './work_space/models/', 'log_path': './work_space/log', 'save_path': './work_space/save', 'input_size': [112, 112], 'embedding_size': 512, 'use_mobilfacenet': False, 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cpu'), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'small_vgg', 'smallvgg_folder': '../data/small_vgg', 'vgg_folder': '../data/faces_vgg2_112x112', 'batch_size': 32, 'lr': 0.001, 'milestones': [12, 15, 18], 'momentum': 0.9, 'pin_memory': True, 'num_workers': 3, 'ce_loss': CrossEntropyLoss(), 'threshold': 1.5}


In [228]:
if conf.use_mobilfacenet:
    model = MobileFaceNet(conf.embedding_size).to(conf.device)
    print('MobileFaceNet model generated')
else:
    model = Backbone(conf.net_depth, conf.drop_ratio, conf.net_mode).to(conf.device)
    print('{}_{} model generated'.format(conf.net_mode, conf.net_depth))

ir_se_50 model generated


In [229]:
if not inference:
    milestones = conf.milestones
    loader, class_num = get_train_loader(conf)   
    print('loader',loader)

    writer = SummaryWriter(conf.log_path)
    step = 0
    head = Arcface(embedding_size=conf.embedding_size, classnum=class_num).to(conf.device)

    print('two model heads generated')

class_num 3
loader <torch.utils.data.dataloader.DataLoader object at 0x000001DCE93C8488>
two model heads generated


In [230]:
paras_only_bn, paras_wo_bn = separate_bn_paras(model)
print(len(paras_only_bn), len(paras_wo_bn))

108 635


In [231]:
if conf.use_mobilfacenet:
    optimizer = optim.SGD([
                        {'params': paras_wo_bn[:-1], 'weight_decay': 4e-5},
                        {'params': [paras_wo_bn[-1]] + [head.kernel], 'weight_decay': 4e-4},
                        {'params': paras_only_bn}
                    ], lr = conf.lr, momentum = conf.momentum)
else:
    optimizer = optim.SGD([
                        {'params': paras_wo_bn + [head.kernel], 'weight_decay': 5e-4},
#                                     {'params': paras_only_bn}
                    ], lr = conf.lr, momentum = conf.momentum)
print(optimizer)
#             self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=40, verbose=True)

print('optimizers generated')    
#             

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
optimizers generated


In [232]:
model.train()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): bottleneck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): PReLU(num_parameters=64)
        (3)

In [233]:
print(len(loader))  # 데이터를 32개씩 7번 갖고오기
# self.board_loss_every = len(self.loader)//100
# self.evaluate_every = len(self.loader)//10
# self.save_every = len(self.loader)//5
board_loss_every = len(loader)
evaluate_every = len(loader)
save_every = len(loader)
#             self.agedb_30, self.cfp_fp, self.lfw, self.agedb_30_issame, self.cfp_fp_issame, self.lfw_issame = get_val_data(self.loader.dataset.root.parent)
lfw, lfw_issame = get_val_data(conf.smallvgg_folder)
print(lfw.shape, len(lfw_issame))

#         else:
threshold = conf.threshold
print(threshold)

19
(198, 3, 112, 112) 99
1.5


In [234]:
def save_state(conf, accuracy, to_save_folder=False, extra=None, model_only=False):
    if to_save_folder:
        save_path = conf.save_path
    else:
        save_path = conf.model_path
#     torch.save(
#         model.state_dict(), save_path('model_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))
#     if not model_only:
#         torch.save(
#             head.state_dict(), save_path('head_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))
#         torch.save(
#             optimizer.state_dict(), save_path('optimizer_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))
    torch.save(
        model.state_dict(), 
        os.path.join(save_path.replace('\\\','/'),'model_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))

    if not model_only:
        torch.save(
            head.state_dict(),
            os.path.join(save_path,'head_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))
        torch.save(
            optimizer.state_dict(), 
            os.path.join(save_path,'optimizer_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))


def load_state(conf, fixed_str, from_save_folder=False, model_only=False):
    if from_save_folder:
        save_path = conf.save_path
    else:
        save_path = conf.model_path            
    model.load_state_dict(torch.load(save_path/'model_{}'.format(fixed_str)))
    if not model_only:
        head.load_state_dict(torch.load(save_path/'head_{}'.format(fixed_str)))
        optimizer.load_state_dict(torch.load(save_path/'optimizer_{}'.format(fixed_str)))        

SyntaxError: invalid syntax (<ipython-input-234-ddf201de0bdf>, line 15)

In [235]:
def board_val(db_name, accuracy, best_threshold, roc_curve_tensor, step):
    """log 남기기"""
    writer.add_scalar('{}_accuracy'.format(db_name), accuracy, step)
    writer.add_scalar('{}_best_threshold'.format(db_name), best_threshold, step)
    writer.add_image('{}_roc_curve'.format(db_name), roc_curve_tensor, step)
#         self.writer.add_scalar('{}_val:true accept ratio'.format(db_name), val, self.step)
#         self.writer.add_scalar('{}_val_std'.format(db_name), val_std, self.step)
#         self.writer.add_scalar('{}_far:False Acceptance Ratio'.format(db_name), far, self.step)



def find_lr(
            conf,
            init_value=1e-8,
            final_value=10.,
            beta=0.98,
            bloding_scale=3.,
            num=None):
    if not num:
        num = len(self.loader)
    mult = (final_value / init_value)**(1 / num)
    lr = init_value
    for params in optimizer.param_groups:
        params['lr'] = lr
    model.train()
    avg_loss = 0.
    best_loss = 0.
    batch_num = 0
    losses = []
    log_lrs = []
    for i, (imgs, labels) in tqdm(enumerate(loader), total=num):

        imgs = imgs.to(conf.device)
        labels = labels.to(conf.device)
        batch_num += 1          

        optimizer.zero_grad()

        embeddings = model(imgs)
        thetas = head(embeddings, labels)
        loss = conf.ce_loss(thetas, labels)          

        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1 - beta) * loss.item()
        writer.add_scalar('avg_loss', avg_loss, batch_num)
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        writer.add_scalar('smoothed_loss', smoothed_loss,batch_num)
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > bloding_scale * best_loss:
            print('exited with best_loss at {}'.format(best_loss))
            plt.plot(log_lrs[10:-5], losses[10:-5])
            return log_lrs, losses
        #Record the best loss
        if smoothed_loss < best_loss or batch_num == 1:
            best_loss = smoothed_loss
        #Store the values
        losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        writer.add_scalar('log_lr', math.log10(lr), batch_num)
        #Do the SGD step
        #Update the lr for the next step

        loss.backward()
        optimizer.step()

        lr *= mult
        for params in optimizer.param_groups:
            params['lr'] = lr
        if batch_num > num:
            plt.plot(log_lrs[10:-5], losses[10:-5])
            return log_lrs, losses    

In [236]:
def schedule_lr():
    print('self.optimizer.param_groups',optimizer.param_groups)
    for params in optimizer.param_groups:                 
        params['lr'] /= 10
    print(optimizer)

def infer( conf, faces, target_embs, tta=False):
    '''
    faces : list of PIL Image
    target_embs : [n, 512] computed embeddings of faces in facebank
    names : recorded names of faces in facebank
    tta : test time augmentation (hfilp, that's all)
    '''
    embs = []
    for img in faces:
        if tta:
            mirror = trans.functional.hflip(img)
            emb = model(conf.test_transform(img).to(conf.device).unsqueeze(0))
            emb_mirror = self.model(conf.test_transform(mirror).to(conf.device).unsqueeze(0))
            embs.append(l2_norm(emb + emb_mirror))
        else:                        
            embs.append(model(conf.test_transform(img).to(conf.device).unsqueeze(0)))
    source_embs = torch.cat(embs)

    diff = source_embs.unsqueeze(-1) - target_embs.transpose(1,0).unsqueeze(0)
    dist = torch.sum(torch.pow(diff, 2), dim=1)
    minimum, min_idx = torch.min(dist, dim=1)
    min_idx[minimum > self.threshold] = -1 # if no match, set idx to -1
    return min_idx, minimum               

### train()

####  self_evaluate(conf,lfw, lfw_issame)

In [237]:
def self_evaluate(conf, carray, issame, nrof_folds = 5, tta = False):
    print('*******start self_evaluate*******')
    model.eval()
    idx = 0
    embeddings = np.zeros([len(carray), conf.embedding_size])
    print('embeddings.shape', embeddings.shape)
    print('len(carray)', len(carray))
    
    with torch.no_grad():
       
        while idx + conf.batch_size <= len(carray):
            print('idx', idx)
            print('idx + conf.batch_size', idx + conf.batch_size)
            batch = torch.tensor(carray[idx:idx + conf.batch_size])
            print('-batch.shape', batch.shape)
            
             # tta : test time augmentation (hfilp, that's all)
            if tta:
                fliped = hflip_batch(batch)
                emb_batch = model(batch.to(conf.device)) + model(fliped.to(conf.device))
                embeddings[idx:idx + conf.batch_size] = l2_norm(emb_batch)
            else:
                print('-model(batch.to(conf.device)).cpu().shape', model(batch.to(conf.device)).cpu().shape)
                embeddings[idx:idx + conf.batch_size] = model(batch.to(conf.device)).cpu()
            idx += conf.batch_size
            
        if idx < len(carray):
            batch = torch.tensor(carray[idx:])            
            if tta:
                fliped = hflip_batch(batch)
                emb_batch = model(batch.to(conf.device)) + model(fliped.to(conf.device))
                embeddings[idx:] = l2_norm(emb_batch)
            else:
                embeddings[idx:] = model(batch.to(conf.device)).cpu()
    tpr, fpr, accuracy, best_thresholds = evaluate(embeddings, issame, nrof_folds)
    print('tpr',tpr.shape, tpr[0]) 
    print('fpr',fpr)
    print('accuracy',accuracy), 
    print('best_thresholds', best_thresholds)
    buf = gen_plot(fpr, tpr)
    roc_curve = Image.open(buf)
    roc_curve_tensor = trans.ToTensor()(roc_curve)
    return accuracy.mean(), best_thresholds.mean(), roc_curve_tensor


#### evaluate(embeddings, issame, nrof_folds)

In [238]:
def evaluate(embeddings, actual_issame, nrof_folds=10,pca=0):
    print('**********evaluate start*******')

    # Calculate evaluation metrics
    print('embeddings.shape',embeddings.shape)  # (198, 512)
    thresholds = np.arange(0, 4, 0.01)
    embeddings1 = embeddings[0::2]   # paris.txt의 0culmns, (99, 512)
    embeddings2 = embeddings[1::2]   # paris.txt의 1culmns, ( 99, 512)
    tpr, fpr, accuracy, best_thresholds = calculate_roc(thresholds, embeddings1, embeddings2,\
                                       np.asarray(actual_issame), nrof_folds=nrof_folds, pca=pca)
    #     thresholds = np.arange(0, 4, 0.001)
    #     val, val_std, far = calculate_val(thresholds, embeddings1, embeddings2,
    #                                       np.asarray(actual_issame), 1e-3, nrof_folds=nrof_folds)
    #     return tpr, fpr, accuracy, best_thresholds, val, val_std, far
    return tpr, fpr, accuracy, best_thresholds

#### calculate_accuracy

In [239]:
def calculate_accuracy(threshold, dist, actual_issame):
#     print('***********calucluate_accuracy start***********')

    predict_issame = np.less(dist, threshold) # dist < threshshold: True
    print('predict_issame.shape',predict_issame.shape, predict_issame[0])   # (80, )
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    print('tp.shape', tp.shape)
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))

    tpr = 0 if (tp + fn == 0) else float(tp) / float(tp + fn)
    fpr = 0 if (fp + tn == 0) else float(fp) / float(fp + tn)
    acc = float(tp + tn) / dist.size
    return tpr, fpr, acc



#### calculate_roc(thresholds, embeddings1, embeddings2,np.asarray(actual_issame), nrof_folds=nrof_folds, pca=pca)

In [240]:
def calculate_roc(thresholds, embeddings1, embeddings2, actual_issame, nrof_folds=10, pca=0):
    print('*********calculate_roc start**********')
#     print('thresholds',thresholds)   # 0.00 ~ 0.39
    assert (embeddings1.shape[0] == embeddings2.shape[0])
    assert (embeddings1.shape[1] == embeddings2.shape[1])
    print('len(actual_issame)',len(actual_issame))              # len(issame)
    print('embeddings1.shape[0]',embeddings1.shape[0])          # len(issame)
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    print('nrof_pairs',nrof_pairs)                              #len(issame)
    nrof_thresholds = len(thresholds)        # 400
    print('nrof_thresholds',nrof_thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=False)

    tprs = np.zeros((nrof_folds, nrof_thresholds))
    print('tprs.shape',tprs.shape)                        # (5, 400)
    fprs = np.zeros((nrof_folds, nrof_thresholds))
    accuracy = np.zeros((nrof_folds))                    # (5,1)
    best_thresholds = np.zeros((nrof_folds))   # ( 5,1)
    indices = np.arange(nrof_pairs)   # 0 ~ 98
    print('indices.shape',indices )   # (99,)
    # print('pca', pca)
 
    if pca == 0:
        diff = np.subtract(embeddings1, embeddings2)
        print('diff', diff.shape)     #(99, 512)
        dist = np.sum(np.square(diff), 1)
        print('dist', dist.shape, dist[0])   # (99,) 0.050621911058111535
        
    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        print('fold_idx', fold_idx)          # 0,1,2,3,4
        print('train_set', train_set.shape)  #(79, )
        print('test_set', test_set.shape)    # (20, )
        if pca > 0:
            print('doing pca on', fold_idx)
            embed1_train = embeddings1[train_set]
            embed2_train = embeddings2[train_set]
            _embed_train = np.concatenate((embed1_train, embed2_train), axis=0)
            print('_embed_train.shape',_embed_train.shape)
            pca_model = PCA(n_components=pca)
            pca_model.fit(_embed_train)
            embed1 = pca_model.transform(embeddings1)
            embed2 = pca_model.transform(embeddings2)
            embed1 = sklearn.preprocessing.normalize(embed1)
            embed2 = sklearn.preprocessing.normalize(embed2)
            print('embde1.shape',embed1.shape)
            print('embde2.shape',embed2.shape)
            diff = np.subtract(embed1, embed2)
            dist = np.sum(np.square(diff), 1)

        # Find the best threshold for the fold
        # 각 thresholds별로 모든 accuracy를 계산해서 가장 높은 accuracy를 가진 threshold값을 찾음.
        acc_train = np.zeros((nrof_thresholds))  # (400, )
        for threshold_idx, threshold in enumerate(thresholds):
            print('threshold_idx',threshold_idx)   # (0 ~ 400)
            print('threshold',threshold)        # (0.0 ~ 3.00)
            tpr_train, fpr_train, acc_train[threshold_idx] = calculate_accuracy(threshold, dist[train_set], actual_issame[train_set])
            print('acc_train[threshold_idx]',acc_train[threshold_idx])
        
        best_threshold_index = np.argmax(acc_train)
#         print('best_threshold_index', best_threshold_index, acc_train[best_threshold_index])
        best_thresholds[fold_idx] = thresholds[best_threshold_index]
    
    
        # test
        for threshold_idx, threshold in enumerate(thresholds):
            tprs[fold_idx, threshold_idx], fprs[fold_idx, threshold_idx], _ = calculate_accuracy(threshold,
                                                                                                 dist[test_set],
                                                                                                 actual_issame[
                                                                                                     test_set])
        _, _, accuracy[fold_idx] = calculate_accuracy(thresholds[best_threshold_index], dist[test_set],
                                                      actual_issame[test_set])

    tpr = np.mean(tprs, 0)
    fpr = np.mean(fprs, 0)
    return tpr, fpr, accuracy, best_thresholds

#### train()

In [241]:
def train(conf, epochs) :
    epochs = 1
    step = 1
    model.train()

    running_loss = 0.  
    for e in range(epochs):
        print('epoch {} started'.format(e))

        # schdule_lr: lr을 1/10으로 나누기
        if e == milestones[0]:
            schedule_lr()
        if e == milestones[1]:
            schedule_lr()      
        if e == milestones[2]:
            schedule_lr()    


        for imgs, labels in tqdm(iter(loader)):
            print(step,'step training start!!!')
            imgs = imgs.to(conf.device)
    #         print('len(imgs)',len(imgs))
            labels = labels.to(conf.device)
    #         print('labels[0]',labels[0])
            optimizer.zero_grad()
            embeddings = model(imgs)
    #         print('embeddings.shape',embeddings.shape)
            thetas = head(embeddings, labels)
            loss = conf.ce_loss(thetas, labels)
            loss.backward()
            running_loss += loss.item()
            optimizer.step()
    #             if step % board_loss_every == 0 and step != 0:
            loss_board = running_loss / board_loss_every
            writer.add_scalar('train_loss', loss_board, step)
            running_loss = 0.

            print('step', step)
            print('ebaluate_every', evaluate_every)
            print('save_every', save_every)
            print('board_loss_every',board_loss_every)

            if step % board_loss_every == 1 and step != 0:
                loss_board = running_loss / board_loss_every
                print('loss_board',loss_board)
                writer.add_scalar('train_loss', loss_board, step)
                running_loss = 0.


            if step % evaluate_every == 1 and step != 0:
    #             accuracy, best_threshold, roc_curve_tensor = self.evaluate(conf, self.agedb_30, self.agedb_30_issame)
    #             self.board_val('agedb_30', accuracy, best_threshold, roc_curve_tensor)
                accuracy, best_threshold, roc_curve_tensor = self_evaluate(conf,lfw, lfw_issame)

                print('accuracy:', accuracy)   # (5,1)
                print('best_threshold', best_threshold)  # thresholde의 평군
                print('roc_curve_tensor.shape',roc_curve_tensor.shape)  # 3, 288, 432]
                board_val('small_lfw', accuracy, best_threshold, roc_curve_tensor, 1)
    #             accuracy, best_threshold, roc_curve_tensor = self.evaluate(conf, self.cfp_fp, self.cfp_fp_issame)
    #             self.board_val('cfp_fp', accuracy, best_threshold, roc_curve_tensor)
                model.train()
            if step % save_every == 1 and step != 0:
                save_state(conf, accuracy, to_save_folder = False, extra = 'training')

            step += 1

    save_state(conf, accuracy, to_save_folder=True, extra='final')

In [242]:
train(conf, epochs = 1)

epoch 0 started


  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

1 step training start!!!
step 1
ebaluate_every 19
save_every 19
board_loss_every 19
loss_board 0.0
*******start self_evaluate*******
embeddings.shape (198, 512)
len(carray) 198
idx 0
idx + conf.batch_size 32
-batch.shape torch.Size([32, 3, 112, 112])
-model(batch.to(conf.device)).cpu().shape torch.Size([32, 512])
idx 32
idx + conf.batch_size 64
-batch.shape torch.Size([32, 3, 112, 112])
-model(batch.to(conf.device)).cpu().shape torch.Size([32, 512])
idx 64
idx + conf.batch_size 96
-batch.shape torch.Size([32, 3, 112, 112])
-model(batch.to(conf.device)).cpu().shape torch.Size([32, 512])
idx 96
idx + conf.batch_size 128
-batch.shape torch.Size([32, 3, 112, 112])
-model(batch.to(conf.device)).cpu().shape torch.Size([32, 512])
idx 128
idx + conf.batch_size 160
-batch.shape torch.Size([32, 3, 112, 112])
-model(batch.to(conf.device)).cpu().shape torch.Size([32, 512])
idx 160
idx + conf.batch_size 192
-batch.shape torch.Size([32, 3, 112, 112])
-model(batch.to(conf.device)).cpu().shape torch.S

predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9493670886075949
threshold_idx 121
threshold 1.21
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9493670886075949
threshold_idx 122
threshold 1.22
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9493670886075949
threshold_idx 123
threshold 1.23
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9493670886075949
threshold_idx 124
threshold 1.24
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9746835443037974
threshold_idx 125
threshold 1.25
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9746835443037974
threshold_idx 126
threshold 1.26
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9746835443037974
threshold_idx 127
threshold 1.27
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 0.9746835443037974
threshold_idx 128
threshold 1.28
predict_issame.shape (79,) True


threshold_idx 197
threshold 1.97
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 198
threshold 1.98
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 199
threshold 1.99
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 200
threshold 2.0
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 201
threshold 2.0100000000000002
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 202
threshold 2.02
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 203
threshold 2.0300000000000002
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 204
threshold 2.04
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 205
threshold 2.05
predict_issame.shape (79,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 206
t

...

acc_train[threshold_idx] 1.0
threshold_idx 376
threshold 3.7600000000000002
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 377
threshold 3.77
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 378
threshold 3.7800000000000002
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 379
threshold 3.79
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 380
threshold 3.8000000000000003
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 381
threshold 3.81
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 382
threshold 3.8200000000000003
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 383
threshold 3.83
predict_issame.shape (80,) True
tp.shape ()
acc_train[threshold_idx] 1.0
threshold_idx 384
threshold 3.84
predict_issame.shape (80,) True
tp

tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape

  0%|                                                                                           | 0/19 [02:22<?, ?it/s]


predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True
tp.shape ()
predict_issame.shape (19,) True

OSError: [Errno 22] Invalid argument: './work_space/models/model_2020-07-23-15-04_accuracy:0.99_step:0_training.pth'

In [221]:
os.path.join('./work_space/models/', 'model_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), 0.99, 1, 'final'))

'./work_space/models/model_2020-07-23-15-01_accuracy:0.99_step:1_final.pth'

In [249]:
a = [1.1, 1.2, 1.2]
sum(a)/3

1.1666666666666667

In [ ]:
import os

In [91]:
conf.data_path = Path('../data')
conf.work_path = Path('./work_space/')
conf.model_path = Path(conf.work_path/'models')
accuracy = 0.99
extra = 'final'
save_path = conf.model_path
print(save_path)
torch.save( obj =model.state_dict(),
           f =save_path('model_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra)))

work_space\models


TypeError: 'WindowsPath' object is not callable

In [59]:
# a = PureWindowsPath('work_space')
a = PurePath('work_space')

b = a/'models'
print(b)

work_space\models


In [60]:
b('a.txt')

TypeError: 'PureWindowsPath' object is not callable

In [55]:
save_path('model_{}_accuracy:{}_step:{}_{}.pth'.format(get_time(), accuracy, step, extra))

NameError: name 'save_path' is not defined

In [85]:
os.path.isdir(b)

False

In [92]:
model.train()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): bottleneck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): PReLU(num_parameters=64)
        (3)

# imgs dir 풀러오기

In [243]:
path = '../data/kface/1.ordinary_test'

In [245]:
len(os.listdir(path))

400